In [ ]:
# Reference
# https://yurimkoo.github.io/python/2019/09/14/connect-db-with-python.html

import pandas as pd
import requests
import pymysql
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
from functions.pubgapi import get_tournament_info
from functions.pubgapi import get_match_info
from functions.pubgapi import get_match_participant_single

# Load api key & RDS info (endpoint="rds 엔드포인트", dbname="db 이름", username="마스터 사용자 이름", password="rds 비밀번호")
load_dotenv()
api_key = os.environ.get("API_KEY")
endpoint = os.environ.get("AWS_endpoint")
dbname = os.environ.get('AWS_dbname')
username = os.environ.get("AWS_username")
password = os.environ.get("AWS_password")

In [ ]:
# Tournament id 값 조회
tournament_info = get_tournament_info(api_key)
tournament_id = tournament_info["id"]

print(f"{len(tournament_id)} 개의 tournament_id 조회 완료\n")

# tournament_index를 지정하여 id값 출력
idx1 = 0
tournament_index_id = tournament_id[idx1]

cur_match_info = get_match_info(api_key, tournament_index_id)
num_match = len(cur_match_info["matchId"])
print(f"Tournament name: {tournament_index_id} ({num_match} matches)")

# Match participant 조회
idx2 = 2
match_info_id = cur_match_info["matchId"]
match_info_id_single = match_info_id[idx2]
match_participant = get_match_participant_single(api_key, match_info_id_single)
match_participant

In [ ]:
# Kill 수의 합이 0인 경우 제외(연습게임)
if match_participant["kills"].sum() != 0:
    upload_table = match_participant
    upload_table_name = f"Match-{match_info_id_single}"

In [ ]:
# DB와 connect
# create_engine("mysql+pymysql://아이디:"+"암호"+"@주소:포트/데이터베이스이름?charset=utf8", encoding='utf-8')
engine = create_engine("mysql+pymysql://admin:"+password+"@"+endpoint+":3306/"+dbname+"?charset=utf8", encoding="utf-8")
conn_create = engine.connect()

conn = pymysql.connect(host=endpoint, user=username, password=password, db=dbname, charset='utf8', port=3306)
cur = conn.cursor()

In [ ]:
# AWS RDS에 테이블 create
upload_table.to_sql(name=upload_table_name, con=conn_create, if_exists='replace', index=False)

In [ ]:
# 업로드된 테이블 확인(SQL 사용)
sql = f"SHOW TABLES LIKE '{upload_table_name}';"
cur.execute(sql)
result = cur.fetchall()
print("SHOW TABLES: \n",result)

In [ ]:
# 내용 확인(SELECT *)
sql = f"SELECT * FROM {upload_table_name};"
cur.execute(sql)
result1 = cur.fetchall()
result1 = pd.DataFrame(result1)
print("\ntournament_info(database):\n",result1)

In [ ]:
conn.commit()

# db접속 종료
conn_create.close()
conn.close()